# <center> Feature Engineering </center>

## Problem Statement  
 - The objective of this notebook is to perform feature extraction from the given dataset. Specifically, we aim to create a  Global alignment attempts to align two sequences from end to end and generate a Target Label Feature based on the Pairwise Alignment Score. 

## Feature Creation Steps

### 1. Creating a Global Weighted Alignment Sequence Feature

- By implementing the **needleman_wunsch_similarity** function to compare DNA sequences and calculate the score of paternity based on matching alleles using the Needleman-Wunsch algorithm. 
 
### 2. Generating a Target Label Feature
- By utilizing the Alignment sequence feature to generate a binary target label (0, 1). The label is determined by a predefined threshold based on medical research. 

------

In [10]:
# Constants
DATA_PATH = '../data/processed/1_first_processed_merged_df.pkl'
EXPORT_PATH = '../data/processed/2_second_processed_merged_df.pkl'

In [11]:
import pandas as pd 

## Read data

In [12]:
first_processed_merged_df = pd.read_pickle(DATA_PATH)

In [13]:
first_processed_merged_df_copy = first_processed_merged_df.copy()

In [14]:
first_processed_merged_df_copy.sample(10)

,Name,Full_seq_dna_parent,Full_seq_dna_child,ParentM,ParentF
14974,A10164,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTTATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN
13126,A8964,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN
11764,A8050,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A8050,A1789
19770,A13577,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN
1099,A693,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATCCCCGTG...,A693,A9165
4039,A2669,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN
14951,A10158,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN
8437,A5783,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A5783,A4690
9842,A6720,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A6720,A10576
10659,A7094,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTTATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATAGGAGCTCTGATTCCCGTG...,NaN,NaN


In [15]:
first_processed_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 43784 entries, 0 to 21891
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Name                 43784 non-null  object
 1   Full_seq_dna_parent  43784 non-null  object
 2   Full_seq_dna_child   43784 non-null  object
 3   ParentM              21892 non-null  object
 4   ParentF              21892 non-null  object
dtypes: object(5)
memory usage: 2.0+ MB


------

## **Global Weighted Alignment Sequence Ratio**

### Compares two sequences (alleles) for a paternity test based on the Pairwise Alignment Score

- Global alignment aims to align the entire length of two sequences. It is suitable when the sequences are of similar length and are expected to be homologous throughout.
 

------

In [17]:
from Bio import pairwise2

def needleman_wunsch_similarity(seq1, seq2, match_score=3, mismatch_penalty=-1, gap_penalty=-2):
    alignments = pairwise2.align.globalms(seq1, seq2, match_score, mismatch_penalty, gap_penalty, gap_penalty)
    """
    Calculates the similarity score between two sequences using the Needleman-Wunsch algorithm.

    Parameters:
        seq1 (str): First DNA sequence.
        seq2 (str): Second DNA sequence.
        match_score (int, optional): Score for matches. Defaults to 3.
        mismatch_penalty (int, optional): Penalty for mismatches. Defaults to -1.
        gap_penalty (int, optional): Penalty for gaps. Defaults to -2.

    Returns:
        float: Similarity score between the two sequences.
    """
    if not alignments:
        return 0.0

    best_alignment = alignments[0]  # Highest score alignment 
    alignment_score = best_alignment[2]  # Score of the best alignment
    max_score = max(len(seq1), len(seq2)) * match_score
    similarity_score = alignment_score / max_score
    return similarity_score

# Example usage:
father_sequence = first_processed_merged_df_copy.iloc[20, 1]
child_sequence = first_processed_merged_df_copy.iloc[20, 2]
similarity_score = needleman_wunsch_similarity(father_sequence, child_sequence)
print("Similarity Score: {:.2%}".format(similarity_score))

Similarity Score: 92.20%


How similar is the DNA of siblings?
- The concept of ‘shared’ DNA differs depending on how much DNA is being compared. When comparing our whole genome, all 3 billion letters, we share about 99.9 percent with every other human. However, we can also analyse shared DNA by narrowing our scope and just comparing the 3 million genetic differences we know exist between individuals. Of these 3 million differences, on average we share about 50 percent of those with our full siblings.

- Children inherit half of their DNA from their mother and half from their father. However, unless they are identical twins, siblings won’t inherit exactly the same DNA.
Depending on their biological sex, a parent will produce either a sperm or egg cell by meiosis. Meiosis is a process where a single cell divides twice to produce four cells containing half the original amount of genetic information. At fertilisation, two of these cells (one egg cell and one sperm cell) will unite to form a single cell. This cell will contain a full set of 46 chromosomes, which make up the child’s DNA.


------

In [18]:
# Constants for needleman_wunsch_similarity
# Still point of researching 
# need to be discussed
NEEDLEMAN_SIMILARITY = 0.95

## **Target Label**

In [19]:
def get_target(df):
    df['similarity_score'] = df.apply(lambda row: needleman_wunsch_similarity(row['Full_seq_dna_parent'], row['Full_seq_dna_child']), axis=1)
    df['target'] = (df['similarity_score'] >= NEEDLEMAN_SIMILARITY).astype(int)  # Set target to 1 or 0
    return df

In [9]:
second_processed_merged_df = get_target(first_processed_merged_df_copy)

In [10]:
second_processed_merged_df

,Name,Full_seq_dna_parent,Full_seq_dna_child,ParentM,ParentF,similarity_score,target
0,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0.929000,0
1,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0.929000,0
2,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGACTCCCGTG...,NaN,NaN,0.968000,1
3,A0,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTTCCGTG...,NaN,NaN,0.963667,1
4,A1,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0.939667,0
...,...,...,...,...,...,...,...
21887,A14997,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A14997,A1357,0.960833,1
21888,A14997,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A14997,A4244,0.927000,0
21889,A14997,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A14997,A4244,0.963167,1
21890,A14997,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A14997,A4244,0.957667,1


In [11]:
second_processed_merged_df['target'].value_counts()

target
0    21953
1    21831
Name: count, dtype: int64

In [12]:
second_processed_merged_df.sample(100)

,Name,Full_seq_dna_parent,Full_seq_dna_child,ParentM,ParentF,similarity_score,target
9648,A6586,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A6586,A1958,0.927333,0
3063,A2072,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0.969667,1
20726,A14215,CTCCGTCGACGCTTTAGGGACGTAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0.935167,0
460,A283,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A283,A7752,0.927667,0
16289,A11132,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A11132,A11177,0.968667,1
...,...,...,...,...,...,...,...
5606,A3716,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0.951333,1
1345,A826,CTCCGTCGACGCTTTAGGGACATAGATAGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A826,A4044,0.934333,0
12336,A8419,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,A8419,A6313,0.962833,1
12996,A8885,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,CTCCGTCGACGCTTTAGGGACATAGATGGGAGCTCTGATTCCCGTG...,NaN,NaN,0.968167,1


In [13]:
second_processed_merged_df.to_pickle(EXPORT_PATH)